In [1]:
# install syft if we are on google colab
%load_ext autoreload
%autoreload 2
import syft as sy
print(f"You're running syft version: {sy.__version__}")
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing
from syft.core.tensor.autodp.gamma_tensor import GammaTensor
from tqdm import tqdm
import time

/home/azureuser/.local/share/virtualenvs/python39-2vM0bQN-/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You're running syft version: 0.7.0-beta.20


In [2]:
domain_client = sy.login(email='info@openmined.org', password='changethis',port = 8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!


In [3]:
data = domain_client.datasets[-1]

X_train = data["train_images"]
Y_train = data["train_labels"]

X_dev = data["train_images"]
Y_dev = data["train_labels"]

m,n = X_train.public_shape

In [4]:
# Preprocess the images

X_train = X_train.T
X_train = X_train / 255.0

X_dev = X_dev.T
X_dev = X_dev / 255.0

/home/azureuser/PySyft/packages/syft/src/syft/core/tensor/smpc/utils.py:75: RuntimeWarning: invalid value encountered in true_divide
  res = (getattr(np.empty(x_shape), op_str)(np.empty(y_shape))).shape


In [5]:

def init_params():
    W1 = np.random.rand(6, 4096) - 0.5
    b1 = np.random.rand(6, 1) - 0.5
    W2 = np.random.rand(6, 6) - 0.5
    b2 = np.random.rand(6, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return Z*(Z>0)

def softmax(Z):
    exp_cache = Z.exp()
    inv =  (exp_cache.sum().reciprocal())
    A = exp_cache * inv
    return A
    
def forward_prop(W1, b1, W2, b2, X):    
    Z1 = X.__rmatmul__(W1) + b1
    A1 = ReLU(Z1)
    Z2 = A1.__rmatmul__(W2) + b2
    A2 = softmax(Z2) 
    return Z1, A1, Z2, A2


def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size),Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = Y.one_hot()
    dZ2 = A2 - one_hot_Y
    dW2 = dZ2@(A1.T) * (1/m)
    db2 = dZ2.sum() * (1/m)
    dZ1 = dZ2.__rmatmul__(W2.T) * ReLU_deriv(Z1)
    dW1 = dZ1@(X.T) * (1/m)
    db1 = dZ1.sum()*(1/m)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = (dW1 * alpha - W1) * -1
    b1 = (db1 * alpha - b1) * -1
    W2 = (dW2 * alpha - W2) * -1
    b2 = (db2 * alpha - b2) * -1
    return W1, b1, W2, b2


def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    print("[INFO]: Starting training!\n")
    for i in tqdm(range(iterations)):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        # if i % 40 == 0:
        #     print("Predicition will be availabe in the next iteration.....Thank you for your support.")
        #     # predictions = get_predictions(A2)
        #     # print("Iteration: ", i, " Accuracy : " ,get_accuracy(predictions, Y))
    # return Z1, A1, Z2, A2,W1,W2
    return W1, b1, W2, b2

In [6]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 1)

[INFO]: Starting training!



  0%|          | 0/1 [00:00<?, ?it/s]/home/azureuser/PySyft/packages/syft/src/syft/core/tensor/smpc/utils.py:75: RuntimeWarning: overflow encountered in multiply
  res = (getattr(np.empty(x_shape), op_str)(np.empty(y_shape))).shape
/home/azureuser/PySyft/packages/syft/src/syft/core/tensor/smpc/utils.py:75: RuntimeWarning: invalid value encountered in subtract
  res = (getattr(np.empty(x_shape), op_str)(np.empty(y_shape))).shape
100%|██████████| 1/1 [00:01<00:00,  1.14s/it]


In [ ]:
W1.get_copy()

In [ ]:
b1.get_copy()

In [ ]:
W2.get_copy()

In [ ]:
b2.get_copy()